In [1]:
# Install SpeechRecognition for speech-to-text
!pip install SpeechRecognition

# Install ffmpeg and ffmpeg-python for audio processing
!apt-get install -y ffmpeg
!pip install ffmpeg-python

# Install gTTS for text-to-speech
!pip install gTTS

# Install googletrans for language translation
!pip install googletrans==4.0.0-rc1

# Initialize assistant with API keys
news_api_key = "YOUR_API_KEY"
aviation_api_key = "YOUR_API_KEY"

# Import libraries
import requests
from googletrans import Translator
from gtts import gTTS
from IPython.display import HTML, Audio, display
from google.colab.output import eval_js
from base64 import b64decode
import io
import ffmpeg
import speech_recognition as sr
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 34.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=bccde52f3

In [6]:
# JavaScript to record audio in Colab
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
};

recordButton.innerText = "Recording... press to stop";
navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... please wait!";
  }
}

var data = new Promise(resolve=>{
  recordButton.onclick = ()=>{
    toggleRecording();
    setTimeout(() => {
      resolve(base64data.toString());
    }, 2000);
  }
});
</script>
"""

def save_audio(file_name):
    # Display the JavaScript to start recording
    display(HTML(AUDIO_HTML))
    # Retrieve the audio data
    data = eval_js("data")
    if data is None or ',' not in data:
        print("No audio data was recorded.")
        return None

    # Decode the base64 audio data
    binary = b64decode(data.split(',')[1])

    # Convert the audio to WAV format
    process = (ffmpeg
               .input('pipe:0')
               .output('pipe:1', format='wav')
               .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    # Save the output as a file
    with open(file_name, 'wb') as f:
        f.write(output)
    return file_name

class VirtualAssistant:
    def __init__(self, news_api_key, aviation_api_key):
        # Initialize translator, API keys, and recognizer
        self.translator = Translator()
        self.news_api_key = news_api_key
        self.aviation_api_key = aviation_api_key
        self.recognizer = sr.Recognizer()
        # Define task descriptions for identifying user commands
        self.task_descriptions = {
            'translate': "Translate English text to Arabic",
            'news': "Retrieve headline news",
            'flight': "Retrieve flight details",
            'read_headlines': "Read headlines aloud"
        }

    def speak(self, text, lang='en'):
        """Convert text to speech, print it, and play it."""
        print("Baymax says:", text)  # Print what Baymax is saying
        tts = gTTS(text=text, lang=lang)
        tts.save("output.mp3")
        display(Audio("output.mp3"))

    def listen(self):
        """Listen to the user's input through the microphone."""
        print("Please say your request.")
        save_audio("user_audio.wav")
        with sr.AudioFile("user_audio.wav") as source:
            audio = self.recognizer.record(source)
            try:
                text = self.recognizer.recognize_google(audio)
                print(f"You said: {text}")
                return text
            except sr.UnknownValueError:
                print("Sorry, I could not understand the audio.")
                self.speak("Sorry, I could not understand the audio.")
            except sr.RequestError:
                print("Could not request results from Google Speech Recognition service.")
                self.speak("Could not request results from Google Speech Recognition service.")
        return None

    def translate_to_arabic(self):
        """Translate user input from English to Arabic."""
        self.speak("Please say the text you want to translate to Arabic.")
        text_to_translate = self.listen()
        if text_to_translate:
            translated = self.translator.translate(text_to_translate, src='en', dest='ar')
            self.speak("Here is the translation in Arabic.", lang='en')
            self.speak(translated.text, lang='ar')  # speak the Arabic translation
            return None

    def get_headlines(self):
        """Retrieve headline news."""
        url = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={self.news_api_key}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            headlines = [article['title'] for article in data['articles']]
            return headlines
        else:
            print("Failed to retrieve headlines.")
            return []

    def get_flight_details(self):
        """Retrieve flight details based on spoken flight number."""
        self.speak("Please say the flight number.")
        flight_number = self.listen()  # Listen for flight number

        if flight_number:
            # Process the flight number to standardize the format
            # Convert it to uppercase and remove any spaces
            flight_number = flight_number.upper().replace(" ", "")

            # Call the aviation API with the provided flight number
            url = f'http://api.aviationstack.com/v1/flights?access_key={self.aviation_api_key}&flight_iata={flight_number}'
            response = requests.get(url)

            if response.status_code == 200:
                flight_data = response.json()
                if flight_data['data']:
                    flight = flight_data['data'][0]
                    details = {
                        "Airline": flight['airline']['name'],
                        "Flight Number": flight['flight']['iata'],
                        "Departure Airport": flight['departure']['airport'],
                        "Arrival Airport": flight['arrival']['airport'],
                        "Departure Time": flight['departure']['estimated'],
                        "Arrival Time": flight['arrival']['estimated'],
                        "Flight Status": flight['flight_status']
                    }
                    # Combine details into a single text for output
                    details_text = "\n".join([f"{key}: {value}" for key, value in details.items()])
                    self.speak(f"Here are the flight details:\n{details_text}")  # Read all details at once
                else:
                    self.speak("No flight data available for this flight number.")
            else:
                self.speak("Failed to retrieve flight details.")
        else:
            self.speak("I couldn't hear the flight number clearly. Please try again.")

    def read_headlines_aloud(self, headlines):
        """Read news headlines aloud."""
        if not headlines:
            print("No headlines to read.")
            return
        combined_text = " . ".join(headlines)
        self.speak(combined_text)

    def identify_task(self, input_text):
        """Identify and perform a task based on user input."""
        # List task descriptions and task names for comparison
        descriptions = list(self.task_descriptions.values())
        task_names = list(self.task_descriptions.keys())

        # Use TF-IDF to transform input text and task descriptions for similarity comparison
        vectorizer = TfidfVectorizer().fit_transform([input_text] + descriptions)
        cosine_similarities = cosine_similarity(vectorizer[0:1], vectorizer[1:]).flatten()

        # Find the index of the most similar task description
        most_similar_index = cosine_similarities.argmax()
        selected_task = task_names[most_similar_index]

        # Execute the identified task based on the similarity results
        if selected_task == 'translate':
            return self.translate_to_arabic()
        elif selected_task == 'news':
            return self.get_headlines()
        elif selected_task == 'flight':
            # Call the flight details function with voice input
            return self.get_flight_details()
        elif selected_task == 'read_headlines':
            # Get and read the news headlines aloud
            headlines = self.get_headlines()
            return self.read_headlines_aloud(headlines)
        else:
            # Print message if no matching task was found
            print("No matching task found.")
            return None

# Initialize assistant with API keys
assistant = VirtualAssistant(news_api_key, aviation_api_key)

# Welcoming the user
assistant.speak("Hello Shahad, I am your virtual assistant Baymax. How can I help you today?")


# Main interaction loop
while True:
    # Listen to user's question
    user_input = assistant.listen()

    # Check if the user wants to exit
    if user_input and user_input.lower() in ["no thank you", "exit"]:
        assistant.speak("Thank you, Shahad! Wishing you a great day from your friendly virtual assistant, Baymax.")
        break

    # Process the user's input if it's not an exit command
    elif user_input:
        result = assistant.identify_task(user_input)

        # Respond based on the task result type
        if isinstance(result, list):
            print("Headlines:", result)
            assistant.speak("Here are the latest headlines.")
        elif isinstance(result, dict):
            print("Flight Details:", result)
            assistant.speak("Here are the flight details.")
        elif isinstance(result, str):
            print("Translated Text:", result)
            assistant.speak(result)

    # After responding, ask if the user has more questions
    while True:
        assistant.speak("Do you have any other questions?")
        follow_up = assistant.listen()

        if follow_up and follow_up.lower() in ["no", "no thank you", "exit"]:
            assistant.speak("Thank you, Shahad! Wishing you a great day from your friendly virtual assistant, Baymax.")
            break  # Break out of the inner loop to stop asking for more questions
        elif follow_up and follow_up.lower() in ["yes", "yes i have"]:
            assistant.speak("Go ahead, what is your question?")
            break  # Return to the main loop to ask the new question

    # Check if we should exit the program completely after inner loop
    if follow_up and follow_up.lower() in ["no", "no thank you", "exit"]:
        break  # Exit the main loop if user is done

Baymax says: Hello Shahad, I am your virtual assistant Baymax. How can I help you today?


Please say your request.


You said: hello Baymax could you get the flight details for a certain flight
Baymax says: Please say the flight number.


Please say your request.


You said: KU 101
Baymax says: Here are the flight details:
Airline: Kuwait Airways
Flight Number: KU101
Departure Airport: Kuwait International
Arrival Airport: Heathrow
Departure Time: 2024-12-09T10:00:00+00:00
Arrival Time: 2024-12-09T13:45:00+00:00
Flight Status: landed


...

Please say your request.


You said: can you read the news headlines for me
Baymax says: Sam Darnold’s five TD passes lead Vikings to 42-21 win over Kirk Cousins’ Falcons - Star Tribune . College Football Playoff bracket: Oregon, Georgia, Boise State, Arizona State earn byes; SMU in over Alabama - CBS Sports . Biden offers a blueprint for US support in Syria, announces airstrikes against ISIS targets - CNN . Syria's Assad is in Moscow after deal on military bases: Russian state media - Reuters . 'The mother of all bubbles' in the US is sucking money away from the rest of the world, market expert says - Fortune . Dolphins knock off Jets 32-26 in overtime - NBC Sports . Israeli military enters Golan Heights as Netanyahu warns off Syrian militants - POLITICO Europe . Fortnite OG Has Up to 90% Bots Per Game, It’s Claimed - Insider Gaming . The end of an era: How Taylor Swift boosted the US economy - CNN . Katie Holmes slams report claiming daughter Suri’s trust fund from Tom Cruise has ‘kicked in’ - Page Six . Trump

Baymax says: Do you have any other questions?


Please say your request.


You said: no
Baymax says: Thank you, Shahad! Wishing you a great day from your friendly virtual assistant, Baymax.
